Does not work with cuda_ad_rgb

In [47]:
import mitsuba as mi
import drjit as dr

mi.set_variant("llvm_ad_rgb")

In [48]:
class TintedDielectric(mi.BSDF):
    def __init__(self: mi.BSDF, props: mi.Properties) -> None:
        mi.BSDF.__init__(self, props)

        # Read 'eta' and 'tint' properties from 'props'
        self.eta = 1.33
        if props.has_property('eta'):
            self.eta = props['eta']

        self.tint = props['tint']

        # set the BSDF flags
        reflection_flags = mi.BSDFFlags.DeltaReflection | mi.BSDFFlags.FrontSide | mi.BSDFFlags.BackSide
        transmission_flags = mi.BSDFFlags.DeltaTransmission | mi.BSDFFlags.FrontSide | mi.BSDFFlags.FrontSide
        self.m_components = [reflection_flags, transmission_flags]
        self.m_flags = reflection_flags | transmission_flags

    def sample(self: mi.BSDF, ctx: mi.BSDFContext, si: mi.SurfaceInteraction3f, sample1: float, sample2: mi.Point2f, active: bool = True):
        # compute fresnel terms
        cos_theta_i = mi.Frame3f.cos_theta(si.wi)
        r_i, cos_theta_t, eta_it, eta_ti = mi.fresnel(cos_theta_i, self.eta)
        t_i = dr.maximum(1.0 - r_i, 0.0)

        # pick between reflection and transmission
        selected_r = (sample1 <= r_i) & active

        # fill up the BSDFSample struct
        bs = mi.BSDFSample3f()
        bs.pdf = dr.select(selected_r, r_i, t_i)
        bs.sampled_component = dr.select(selected_r, mi.UInt32(0), mi.UInt32(1))
        bs.sampled_type = dr.select(selected_r, mi.UInt32(+mi.BSDFFlags.DeltaReflection), mi.UInt32(+mi.BSDFFlags.DeltaTransmission))

        bs.wo = dr.select(selected_r, mi.reflect(si.wi), mi.refract(si.wi, cos_theta_t, eta_ti))
        bs.eta = dr.select(selected_r, 1.0, eta_ti)

        # for reflection, tint based on the incident angle (more tint at grazing angle)
        value_r = dr.lerp(mi.Color3f(self.tint), mi.Color3f(1.0), dr.clamp(cos_theta_i, 0.0, 1.0))

        # for transmission radiance must be scaled to account for the solid angle compression
        value_t = mi.Color3f(1.0) * dr.sqr(eta_ti)

        value = dr.select(selected_r, value_r, value_t)

        return (bs, value)

    def eval(self: mi.BSDF, ctx: mi.BSDFContext, si: mi.SurfaceInteraction3f, wo: mi.Vector3f, active: bool = True) -> mi.Color3f:
        return 0.0

    def pdf(self: mi.BSDF, ctx: mi.BSDFContext, si: mi.SurfaceInteraction3f, wo: mi.Vector3f, active: bool = True) -> float:
        return 0.0

    def eval_pdf(self: mi.BSDF, ctx: mi.BSDFContext, si: mi.SurfaceInteraction3f, wo: mi.Vector3f, active: bool = True):
        return 0.0, 0.0

    def traverse(self: mi.Object, callback: mi.TraversalCallback) -> None:
        callback.put_parameter('tint', self.tint, mi.ParamFlags.Differentiable)

    def parameters_changed(self: mi.Object, keys) -> None:
        print("🏝️ there is nothing to do here 🏝️")

    def to_string(self):
        return ('MyBSDF[\n'
            '    eta=%s,\n'
            '    tint=%s,\n'
            ']' % (self.eta, self.tint))

In [49]:
mi.register_bsdf("tinted_dielectric", lambda props: TintedDielectric(props))

In [50]:
tinted_dielectric = mi.load_dict({
    'type': 'tinted_dielectric',
    'tint': [0.2, 0.9, 0.2],
    'eta': 1.33
})

tinted_dielectric

MyBSDF[
    eta=1.33,
    tint=[0.2, 0.9, 0.2],
]

In [51]:
scene = mi.load_dict({
    'type': 'scene',
    'integrator': {
        'type': 'path'
    },
    'light': {
        'type': 'constant',
        'radiance': 0.99,
    },
    'sphere' : {
        'type': 'sphere',
        'bsdf': tinted_dielectric
    },
    'sensor': {
        'type': 'perspective',
        'to_world': mi.ScalarTransform4f.look_at(origin=[0, -5, 5],
                                                 target=[0, 0, 0],
                                                 up=[0, 0, 1]),
    }
})

image = mi.render(scene)

mi.Bitmap(image).convert(srgb_gamma=True)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [768, 576],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 5.06 MiB of image data ]
]

In [52]:
params = mi.traverse(scene)
params

SceneParameters[
  -------------------------------------------------------------------------------
  Name                        Flags    Type            Parent
  -------------------------------------------------------------------------------
  light.radiance.value        ∂        Float           UniformSpectrum
  sensor.near_clip                     float           PerspectiveCamera
  sensor.far_clip                      float           PerspectiveCamera
  sensor.shutter_open                  float           PerspectiveCamera
  sensor.shutter_open_time             float           PerspectiveCamera
  sensor.x_fov                         Float           PerspectiveCamera
  sensor.to_world                      Transform4f     PerspectiveCamera
  sphere.to_world                      Transform4f     Sphere
  sphere.bsdf.tint            ∂        Array3f64       BSDF
]

In [53]:
key = 'sphere.bsdf.tint'
params[key] = mi.ScalarColor3f(0.9, 0.2, 0.2)
params.update();

🏝️ there is nothing to do here 🏝️


In [54]:
image = mi.render(scene)

mi.Bitmap(image).convert(srgb_gamma=True)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [768, 576],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 5.06 MiB of image data ]
]